In [32]:
import gc
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('max_columns', None)
import numpy as np
from datetime import datetime
import time

from prepare_data import prepare
from load_data import load

import xgboost as xgb
import lightgbm as lgb
# import catboost as ctb

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, RidgeCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train, test, y, train_dict = load()

100%|████████████████████████████████████████████| 8/8 [00:08<00:00,  1.01s/it]


In [3]:
all_data = prepare(pd.concat([train, test]).reset_index(drop=True), train_dict)

train = all_data.loc[:train.shape[0] -1,:]
test = all_data.loc[train.shape[0]:,:] 

In [24]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1))
    return(rmse)

In [4]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))

In [5]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [7]:
# KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [6]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=5)

In [7]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state=7, nthread=-1)

In [8]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=1000,
                              max_bin=55, bagging_fraction=0.8,
                              bagging_freq=5, feature_fraction=0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=6, min_sum_hessian_in_leaf=11)

In [34]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 4.7273 (4.5322)



In [35]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 4.7281 (4.5338)



In [11]:
# score = rmsle_cv(KRR)
# print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [36]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 2.3446 (0.3917)



In [37]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 2.5654 (0.6163)



In [38]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

LGBM score: 2.4145 (0.5321)



In [7]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions, axis=1)

In [8]:
averaged_models = AveragingModels(models=(lasso, ENet, GBoost))

In [ ]:
score = rmsle_cv(averaged_models)
print("Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [9]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [10]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, model_xgb),
                                                 meta_model = lasso)

In [11]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [12]:
stacked_averaged_models.fit(train.values, y)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y, stacked_train_pred))

1.6197091791981342


In [13]:
model_xgb.fit(train, y)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y, xgb_train_pred))

1.0532774206676123


In [14]:
model_lgb.fit(train, y)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y, lgb_train_pred))

1.8192734224504923


In [21]:
print('RMSLE score on train data:')
print(rmsle(y,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

RMSLE score on train data:
1.5816884415147248


In [22]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [23]:
sub = pd.read_csv('data/sample_submission.csv')
df_sub = pd.DataFrame()
df_sub['id'] = sub['id']
df_sub['revenue'] = ensemble
df_sub.to_csv("submission_ens.csv", index=False)

In [15]:
SEED = 42
K = 10
fold = list(KFold(K, shuffle=True, random_state=SEED).split(train))
np.random.seed(SEED)

In [16]:
def xgb_model(trn_X, trn_y, val_X, val_y, test, verbose):
    
    params = {'objective': 'reg:linear',
              'eta': 0.01, 
              'max_depth': 5,
              'subsample': 0.6,
              'colsample_bytree': 0.7,
              'eval_metrics': 'rmse',
              'seed': SEED,
              'silent': True}
    
    record = dict()
    
    model = xgb.train(params, xgb.DMatrix(trn_X, trn_y), 1000,
                      [(xgb.DMatrix(trn_X, trn_y), 'train'),
                      (xgb.DMatrix(val_X, val_y), 'valid')],
                      verbose_eval=verbose,
                      early_stopping_rounds=200,
                      callbacks=[xgb.callback.record_evaluation(record)])
    
    best_idx = np.argmin(np.array(record['valid']['rmse']))
    
    val_pred = model.predict(xgb.DMatrix(val_X), ntree_limit=model.best_ntree_limit)
    test_pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
    
    return {'val': val_pred, 'test': test_pred, 'error': record['valid']['rmse'][best_idx], 'importance': [i for k, i in model.get_score().items()]}

In [17]:
def lgb_model(trn_x, trn_y, val_x, val_y, test, verbose) :

    params = {'objective':'regression',
         'num_leaves' : 40,
         'min_data_in_leaf' : 20,
         'max_depth' : 5,
         'learning_rate': 0.01,
         'feature_fraction': 0.8,
         'bagging_freq': 1,
         'bagging_fraction': 0.8,
         'bagging_seed': SEED,
         'metric': 'rmse',
         'random_state' : SEED,
         'verbosity': -1}

    record = dict()
    model = lgb.train(params
                      , lgb.Dataset(trn_x, trn_y)
                      , num_boost_round = 10000
                      , valid_sets = [lgb.Dataset(val_x, val_y)]
                      , verbose_eval = verbose
                      , early_stopping_rounds = 200
                      , callbacks = [lgb.record_evaluation(record)]
                     )
    best_idx = np.argmin(np.array(record['valid_0']['rmse']))

    val_pred = model.predict(val_x, num_iteration = model.best_iteration)
    test_pred = model.predict(test, num_iteration = model.best_iteration)
    
    return {'val': val_pred, 'test': test_pred, 'error': record['valid_0']['rmse'][best_idx], 'importance': model.feature_importance('gain')}

In [18]:
from catboost import CatBoostRegressor

def cat_model(trn_x, trn_y, val_x, val_y, test, verbose) :
    
    model = CatBoostRegressor(iterations=10000,
                                 learning_rate=0.01,
                                 depth=5,
                                 eval_metric='RMSE',
                                 colsample_bylevel=0.7,
                                 random_seed=SEED,
                                 bagging_temperature=0.2,
                                 metric_period=None,
                                 early_stopping_rounds=200)
                                  
    model.fit(trn_x, trn_y,
                 eval_set=(val_x, val_y),
                 use_best_model=True,
                 verbose=False)
    
    val_pred = model.predict(val_x)
    test_pred = model.predict(test)
    
    return {'val': val_pred, 'test': test_pred, 'error': model.get_best_score()['validation_0']['RMSE'], 'importance': model.get_feature_importance()}

In [19]:
result_dict = dict()
val_pred = np.zeros(train.shape[0])
test_pred = np.zeros(test.shape[0])
final_err = 0
verbose = False

for i, (trn, val) in enumerate(fold) :
    print(i+1, "fold.    RMSE")
    
    trn_x = train.loc[trn, :]
    trn_y = y[trn]
    val_x = train.loc[val, :]
    val_y = y[val]
    
    fold_val_pred = []
    fold_test_pred = []
    fold_err = []
    
    
    start = datetime.now()
    result = xgb_model(trn_x, trn_y, val_x, val_y, test, verbose)
    fold_val_pred.append(result['val'])
    fold_test_pred.append(result['test'])
    fold_err.append(result['error'])
    print("xgb model.", "{0:.5f}".format(result['error']), '(' + str(int((datetime.now()-start).seconds/60)) + 'm)')
    
    start = datetime.now()
    result = lgb_model(trn_x, trn_y, val_x, val_y, test, verbose)
    fold_val_pred.append(result['val'])
    fold_test_pred.append(result['test'])
    fold_err.append(result['error'])
    print("lgb model.", "{0:.5f}".format(result['error']), '(' + str(int((datetime.now()-start).seconds/60)) + 'm)')
    
    start = datetime.now()
    result = cat_model(trn_x, trn_y, val_x, val_y, test, verbose)
    fold_val_pred.append(result['val'])
    fold_test_pred.append(result['test'])
    fold_err.append(result['error'])
    print("cat model.", "{0:.5f}".format(result['error']), '(' + str(int((datetime.now()-start).seconds/60)) + 'm)')
    
      
    val_pred[val] += np.mean(np.array(fold_val_pred), axis = 0)
    test_pred += np.mean(np.array(fold_test_pred), axis = 0) / K
    final_err += (sum(fold_err) / len(fold_err)) / K
    
    print("---------------------------")
    print("avg   err.", "{0:.5f}".format(sum(fold_err) / len(fold_err)))
    print("blend err.", "{0:.5f}".format(np.sqrt(np.mean((np.mean(np.array(fold_val_pred), axis = 0) - val_y)**2))))
    
    print('')
    
print("fianl avg   err.", final_err)
print("fianl blend err.", np.sqrt(np.mean((val_pred - y)**2)))

boost_pred = np.expm1(test_pred)

1 fold.    RMSE
xgb model. 2.20369 (0m)
lgb model. 2.16956 (0m)
cat model. 2.24732 (0m)
---------------------------
avg   err. 2.20686
blend err. 2.19017

2 fold.    RMSE
xgb model. 2.60680 (0m)
lgb model. 2.58602 (0m)
cat model. 2.63044 (0m)
---------------------------
avg   err. 2.60775
blend err. 2.59303

3 fold.    RMSE
xgb model. 2.30970 (0m)
lgb model. 2.29705 (0m)
cat model. 2.31970 (0m)
---------------------------
avg   err. 2.30882
blend err. 2.29539

4 fold.    RMSE
xgb model. 2.29237 (0m)
lgb model. 2.27811 (0m)
cat model. 2.26792 (0m)
---------------------------
avg   err. 2.27947
blend err. 2.26810

5 fold.    RMSE
xgb model. 2.43281 (0m)
lgb model. 2.44185 (0m)
cat model. 2.37311 (0m)
---------------------------
avg   err. 2.41592
blend err. 2.38865

6 fold.    RMSE
xgb model. 2.33372 (0m)
lgb model. 2.32472 (0m)
cat model. 2.32285 (0m)
---------------------------
avg   err. 2.32710
blend err. 2.31444

7 fold.    RMSE
xgb model. 2.16152 (0m)
lgb model. 2.14261 (0m)
cat mo

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2)

In [21]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

In [22]:
def train_model(X, X_test, y, params=None, folds=folds, model_type='lgb', plot_feature_importance=False, model=None):

    oof = np.zeros(X.shape[0])
    prediction = np.zeros(X_test.shape[0])
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        if model_type == 'sklearn':
            X_train, X_valid = X[train_index], X[valid_index]
        else:
            X_train, X_valid = X.values[train_index], X.values[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=1000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test.values), ntree_limit=model.best_ntree_limit)

        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_squared_error(y_valid, y_pred_valid)
            
            y_pred = model.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='RMSE', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_squared_error(y_valid, y_pred_valid) ** 0.5)
        
        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction
    
    else:
        return oof, prediction

In [30]:
xgb_params = {'eta': 0.01,
              'objective': 'reg:linear',
              'max_depth': 5,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'eval_metric': 'rmse',
              'seed': 11,
              'silent': True}

oof_xgb, prediction_xgb = train_model(train, test, y, params=xgb_params, model_type='xgb', plot_feature_importance=False)

Fold 0 started at Tue Feb 19 14:58:41 2019
[0]	train-rmse:15.3476	valid_data-rmse:15.3523
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[500]	train-rmse:1.75352	valid_data-rmse:2.44288
[1000]	train-rmse:1.41732	valid_data-rmse:2.41006
Stopping. Best iteration:
[1274]	train-rmse:1.28846	valid_data-rmse:2.40256

Fold 1 started at Tue Feb 19 14:58:53 2019
[0]	train-rmse:15.356	valid_data-rmse:15.3165
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[500]	train-rmse:1.7981	valid_data-rmse:2.29623
[1000]	train-rmse:1.45894	valid_data-rmse:2.28266
Stopping. Best iteration:
[942]	train-rmse:1.48938	valid_data-rmse:2.28064

Fold 2 started at Tue Feb 19 14:59:02 2019
[0]	train-rmse:15.3707	valid_data-rmse:15.258
Multiple eval metrics have been passed: 'valid_data-rmse' will be used

In [34]:
model = RidgeCV(alphas=(0.01, 0.1, 1.0, 10.0, 100.0), scoring='neg_mean_squared_error', cv=folds)

oof_ridge, prediction_ridge = train_model(train.fillna(0).values, test.fillna(0).values, y, params=None, model_type='sklearn', model=model)

Fold 0 started at Tue Feb 19 15:00:58 2019
Fold 1 started at Tue Feb 19 15:00:59 2019
Fold 2 started at Tue Feb 19 15:00:59 2019
Fold 3 started at Tue Feb 19 15:01:00 2019
Fold 4 started at Tue Feb 19 15:01:00 2019
CV mean score: 6.1487, std: 6.5981.


In [37]:
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 7,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}

oof_lgb, prediction_lgb = train_model(train,test, y, params=params, model_type='lgb', plot_feature_importance=False)

Fold 0 started at Tue Feb 19 15:07:13 2019
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's rmse: 1.36697	valid_1's rmse: 2.43312
Early stopping, best iteration is:
[831]	valid_0's rmse: 1.46307	valid_1's rmse: 2.43068
Fold 1 started at Tue Feb 19 15:07:16 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[425]	valid_0's rmse: 1.80487	valid_1's rmse: 2.26551
Fold 2 started at Tue Feb 19 15:07:17 2019
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's rmse: 1.36581	valid_1's rmse: 2.38286
Early stopping, best iteration is:
[881]	valid_0's rmse: 1.43339	valid_1's rmse: 2.381
Fold 3 started at Tue Feb 19 15:07:19 2019
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's rmse: 1.40697	valid_1's rmse: 2.23667
Early stopping, best iteration is:
[908]	valid_0's rmse: 1.45622	valid_1's rmse: 2.23403
Fold 4 started at Tue Feb 19 15:07:21 2019
Training until

In [38]:
blend_pred = np.expm1((prediction_lgb + prediction_xgb + prediction_ridge) / 3)

In [43]:
ensemble = stacked_pred*0.75 + boost_pred*0.15 + blend_pred*0.1

In [44]:
sub = pd.read_csv('data/sample_submission.csv')
df_sub = pd.DataFrame()
df_sub['id'] = sub['id']
df_sub['revenue'] = ensemble
df_sub.to_csv("submission_ens.csv", index=False)